In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [2]:
data = pd.read_stata('C:\\Users\\bayle\\Documents\\HW4.dta')
display(data)

,person_ID,year,log_wage
0,4.0,1982,9.615806
1,5.0,1974,9.479604
2,5.0,1975,9.384294
3,5.0,1976,7.228388
4,5.0,1977,9.428673
...,...,...,...
33774,35355.0,1975,9.026418
33775,35355.0,1976,8.780787
33776,35355.0,1977,9.086702
33777,35356.0,1975,8.348064


In [4]:
variables_list = data.columns.tolist()
print(variables_list)

['person_ID', 'year', 'log_wage']


In [75]:
variance_by_year = data.groupby('year')['log_wage'].var()
covariance_across_years = data.pivot_table(index='person_ID', columns='year', values='log_wage').cov()
display(covariance_across_years)
data_moments = covariance_across_years.reset_index()
data_moments = data_moments.iloc[:,1:].values
data_moments = np.tril(data_moments)
data_moments = data_moments.reshape(-1,1)
data_moments = data_moments[data_moments!=0].reshape(-1,1)
display(data_moments)

year,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992
year,,,,,,,,,,,,,,,,,,,
1974,0.380232,0.266168,0.223703,0.232121,0.247309,0.219021,0.192690,0.172943,0.174611,0.186813,0.171124,0.169817,0.186373,0.150952,0.141803,0.177476,0.144341,0.129477,0.109535
1975,0.266168,0.411052,0.269560,0.253488,0.267063,0.234334,0.205848,0.190847,0.198061,0.187549,0.181489,0.159429,0.175009,0.169591,0.154137,0.191548,0.177200,0.153252,0.147261
1976,0.223703,0.269560,0.466307,0.298764,0.262443,0.218654,0.206554,0.179327,0.199280,0.209063,0.195643,0.185495,0.181148,0.182515,0.157680,0.162372,0.176461,0.183787,0.153181
1977,0.232121,0.253488,0.298764,0.546291,0.306649,0.255678,0.238145,0.254721,0.228993,0.215633,0.236753,0.205950,0.230313,0.207437,0.188814,0.206426,0.198869,0.219495,0.166731
1978,0.247309,0.267063,0.262443,0.306649,0.579645,0.314938,0.268837,0.264958,0.234453,0.233428,0.244582,0.213733,0.201749,0.210650,0.173736,0.174617,0.163338,0.143215,0.140366
1979,0.219021,0.234334,0.218654,0.255678,0.314938,0.502633,0.304382,0.266425,0.261171,0.252946,0.260685,0.229166,0.218492,0.196992,0.186591,0.201091,0.176052,0.165536,0.164548
1980,0.192690,0.205848,0.206554,0.238145,0.268837,0.304382,0.509569,0.342281,0.302340,0.297831,0.303705,0.259129,0.251555,0.224739,0.221790,0.222968,0.194946,0.182388,0.200839
1981,0.172943,0.190847,0.179327,0.254721,0.264958,0.266425,0.342281,0.542037,0.350024,0.310555,0.303280,0.266673,0.255760,0.252414,0.247930,0.234822,0.210068,0.193489,0.201162
1982,0.174611,0.198061,0.199280,0.228993,0.234453,0.261171,0.302340,0.350024,0.629157,0.348557,0.348459,0.345709,0.325680,0.278233,0.278888,0.255396,0.245366,0.252007,0.219352


array([[0.38023182],
       [0.26616782],
       [0.4110522 ],
       [0.22370324],
       [0.26955968],
       [0.46630692],
       [0.23212118],
       [0.25348755],
       [0.29876361],
       [0.54629134],
       [0.24730907],
       [0.26706259],
       [0.26244273],
       [0.3066485 ],
       [0.57964453],
       [0.21902123],
       [0.23433364],
       [0.218654  ],
       [0.25567779],
       [0.31493762],
       [0.50263275],
       [0.19269017],
       [0.20584785],
       [0.20655434],
       [0.23814452],
       [0.2688371 ],
       [0.30438187],
       [0.50956911],
       [0.17294339],
       [0.19084692],
       [0.17932698],
       [0.25472087],
       [0.26495758],
       [0.26642481],
       [0.34228084],
       [0.54203681],
       [0.17461068],
       [0.19806132],
       [0.19927958],
       [0.22899258],
       [0.23445257],
       [0.26117134],
       [0.30233959],
       [0.35002439],
       [0.6291566 ],
       [0.18681347],
       [0.18754893],
       [0.209

In [178]:
def simulate_income_data(p, num_individuals, num_periods):

    var_alpha, rho, var_eta, var_epsilon = p

    np.random.seed(4) # VVD
    
    # Simulate individual-specific fixed effects
    individual_fixed_effects = np.random.normal(scale=np.sqrt(var_alpha), size=num_individuals)
    
    # Initialize DataFrame to store simulated income data
    simulated_data = pd.DataFrame(index=range(num_individuals), columns=range(num_periods))
    
    # Simulate income data for each individual
    for i in range(num_individuals):

        # Initialise the autoregressive process
        z_process = 0
        
        # Generate income data for each period
        for t in range(num_periods):
            
            # Simulate shock to the autoregressive process
            eta = np.random.normal(scale=np.sqrt(var_eta))
            
            # Update autoregressive process
            z_process = rho * z_process + eta
            
            # Simulate income for the current period
            income = individual_fixed_effects[i] + z_process + np.random.normal(scale=np.sqrt(var_epsilon))
            
            # Store simulated income in DataFrame
            simulated_data.at[i, t] = income
    
    cov_matrix = simulated_data.cov()

    simulated_moments = cov_matrix.values

    simulated_moments = np.tril(simulated_moments)

    simulated_moments = simulated_moments.reshape(-1,1)

    simulated_moments = simulated_moments[simulated_moments!=0].reshape(-1,1) 

    return simulated_moments

In [179]:
# Simulate income data
params = [0.1,0.8,0.1,0.1]
simulated_moments = simulate_income_data(params, num_individuals=1000, num_periods=19)

In [188]:
def metric(p, data_moments, num_individuals, num_periods):

    simulated_moments = simulate_income_data(p, num_individuals, num_periods)

    distance = simulated_moments - data_moments

    dist = np.dot(distance.T, distance)
    
    return dist

In [189]:
dist = metric(params, data_moments, 1000, 19)
print(dist)

[[2.21436166]]


In [194]:
def SMM(p, num_individuals, num_periods, data_moments):

    result = minimize(metric, p, args=(data_moments, num_individuals, num_periods, ), method='Nelder-Mead')

    matched_coefficients = result.x

    return matched_coefficients

In [195]:
initial_params=[0.5,0.9,0.5,0.5]
initial_params_array = np.array(initial_params)

In [196]:
coeffs = SMM(initial_params_array, 1000, 19, data_moments)
print(coeffs)

C:\Users\bayle\AppData\Local\Temp\ipykernel_13440\972145239.py:8: RuntimeWarning: invalid value encountered in sqrt
  individual_fixed_effects = np.random.normal(scale=np.sqrt(var_alpha), size=num_individuals)
C:\Users\bayle\AppData\Local\Temp\ipykernel_13440\972145239.py:8: RuntimeWarning: invalid value encountered in sqrt
  individual_fixed_effects = np.random.normal(scale=np.sqrt(var_alpha), size=num_individuals)
C:\Users\bayle\AppData\Local\Temp\ipykernel_13440\972145239.py:23: RuntimeWarning: invalid value encountered in sqrt
  eta = np.random.normal(scale=np.sqrt(var_eta))


[0.13801785 0.91608059 0.05728621 0.20410723]
